In [ ]:
import os

import pandas as pd

home = os.getenv("HOME") or os.getenv("USERPROFILE")
desktop = home + "/Desktop"

delays = pd.read_csv(f"{desktop}/data/traffic/delays-merged.csv")
traffic = pd.read_csv(f"{desktop}/data/weather/weather-merged.csv")
routes = pd.read_csv(f"{desktop}/data/gtfs/2025/01/03/routes.csv")

# Przygotowanie danych do analizy
## Pogoda oraz opóźnienia
Przed przystąpieniem do analizy wstępnie oczyściliśmy i znormalizowaliśmy dane. Np. w pliku `delays-merged.csv`, wartości kolumny `Delay` typu `x min` zostały podmienione na `x`, a wartości `x min przed czasem` na `-x`.
Połączyliśmy wszystki zebrane pliki z pogodą oraz opóźnieniami w pojedyncze pliki: `weather-merged.csv` oraz `delays-merged.csv`. Po połączeniu plików, zduplikowane linie zostały usunięte.
Przed przystąpieniem do agregacji danych z feedu GTFS, wpierw musimy zdecydować które dane w nim zawarte są dla nas istotne.

## GTFS
Warszawski GTFS składa się z następujących plików:
### `agency.txt`
Ten plik zawiera dane o agencjach których dotyczy feed, ma raczej charakter licencyjny. Z punktu widzenia analizy opóźnień nie jest to istotne.
### `calendar_dates.txt`
Specyfikacja tego pliku jest dość skomplikowana, tak opisuje to dokumentacja:
```
The calendar_dates.txt table explicitly activates or disables service by date. It may be used in two ways.

Recommended: Use calendar_dates.txt in conjunction with calendar.txt to define exceptions to the default service patterns defined in calendar.txt. If service is generally regular, with a few changes on explicit dates (for instance, to accommodate special event services, or a school schedule), this is a good approach. In this case calendar_dates.service_id is a foreign ID referencing calendar.service_id.
Alternate: Omit calendar.txt, and specify each date of service in calendar_dates.txt. This allows for considerable service variation and accommodates service without normal weekly schedules. In this case service_id is an ID.
```
W warszawskim feedzie nie występuje `calendar.txt`, a więc `calendar_dates.txt` opisuje "wydania" rozkładów jazdy, tj. które dane obowiązują w jakich terminach. Takie dane wydają się istotne, ale z feedu gtfs nie wykorzystujemy danych powiązanych z datami.

### `feed_info.txt`
Kolejny plik o charakterze licencyjnym, zawiera informacje o podmiocie udostępniającym feed GTFS. Nieistotny z punktu widzenia analizy.
### `routes.txt`
Ten plik definiuje linie i charakteryzuje kilka ich podstawowych parametrów. Z punktu widzenia naszej analizy, jedyną przydatną informacją jest tutaj przypisanie `route_type` do `route_id`. Wartości `route_type` występujące w pliku oznaczają:
- `0` - Tramwaj lub kolej krótkodystansowa
- `1` - Metro
- `2` - Kolej długodystansowa
- `3` - Autobus

Nie przydadzą nam się następujące dane:
- `1` - Dane o metrze nie występują ani w tym feedzie gtfs, ani w danych o opóźnieniach

In [ ]:
metro_gtfs = routes[routes["route_type"] == 1]
assert metro_gtfs.empty
metro_delays = delays[delays["Route"].str.startswith("M")]
assert metro_delays.empty

- `2` - dane o opóźnieniach zawierają jedynie dane o WKD, podczas kiedy WKD nie występuje w GTFS

**Wniosek**: Z gtfs można usunąć wszystkie dane powiązane z `route_type == 2`, z delays można usunąć wszystkie dane z `Type == "Pociąg"`
Sam w sobie plik `routes.txt` nie przyda się do dalszej analizy, jedynie do zidentyfikowania danych do wyczyszczenia.

In [ ]:
trains_gtfs = routes[routes["route_type"] == 2]
unique_trains_gtfs = trains_gtfs["route_id"].unique()
print(f"Pociągi w GTFS: {unique_trains_gtfs}")
trains_delays = delays[delays["Type"] == "Pociąg"]
unique_trains_delays = trains_delays["Route"].unique()
print(f"Pociągi w delays: {unique_trains_delays}")

### `shapes.txt`
Opisuje punkty na trasie, definiując tym kształty tras (stąd nazwa). Z punktu widzenia naszej analizy zbędny, współrzędne geograficzne konkretnych przystanków mamy już dostępne w `stops.txt`.
### `stop_times.txt`
Opisuje rozkład jazdy. W większości te dane są nam zbędne, jednak istnieją 2 ciekawe kolumny: `pickup_type` oraz `drop_off_type`. Znaczenia tych wartości (z dokumentacji gtfs):
- `0` - Regularly scheduled pickup/dropoff, czyli zwykłe przystanki
- `1` - No pickup/dropoff available, przykładowo nie można wsiąść na przystanku końcowym albo wysiąść na początkowym
- `2` - Must phone agency to arrange pickup, nie występuje w tym feedzie


In [ ]:
stop_times = pd.read_csv(f"{desktop}/data/gtfs/2025/01/03/stop_times.csv", low_memory=False)
stop_times_must_phone_agency = stop_times[stop_times["pickup_type"] == 2]
assert stop_times_must_phone_agency.empty

- `3` - Must coordinate with driver to arrange pickup/dropoff, w ten sposób oznaczane są przystanki na żądanie (obie kolumny: `pickup_type`, `drop_off_type` mają wtedy wartość `3`). \
Analiza wpływu obecności przystanków na żądanie na trasie na opóźnienia może być interesująca, a więc zostawiamy te dane.
### `stops.txt`
Zawiera podstawowe informacje o przystankach - id, nazwa, współrzędne geograficzne.
Powiązanie nazwy przystanku z fizyczną lokalizacją może być przydatne np. do stworzenia heatmapy opóźnień.
### `trips.csv`
Opisuje konkretne przejazdy po liniach zdefiniowanych w `routes.txt`. Jako że nie korzystamy dalej z pliku `routes.txt`, to `trips.txt` będący jego rozszerzeniem również się nam nie przyda.

## Podsumowanie wyboru plików
Do analizy wykorzystane zostaną następujące pliki:
- `weather-merged.csv`
- `delays-merged.csv`
- `gtfs/stop_times.txt`
- `gtfs/stops.txt`

`gtfs/routes.txt` zostaną wykorzystane jedynie do rozpoznania, które dane w `stops.txt` oraz `stop_times.txt` dotyczą metra lub pociągów.

Następnym krokiem jest przegląd każdego z plików i zdecydowanie, czy wszystkie dane są nam potrzebne. \
Pliki są dość duże, więc każda usunięta kolumna znacznie przyspieszy jakiekolwiek operacje wykonywane na danych.

## Przegląd poszczególnych plików
### `weather-merged.csv`
- `id_stacji` - wszystkie pomiary są dokonywane z tej samej stacji pomiarowej, każda wartość w tej kolumnie to 12375, więc jest ona zbędna.
- `stacja` - tak samo jak wyżej, z tym że tutaj każda wartość to "Warszawa"
- `temperatura` - może mieć istotny wpływ na warunki drogowe, zostawiamy
- `predkosc_wiatru` - może mieć istotny wpływ na warunki drogowe, zostawiamy
- `kierunek_wiatru` - to w którą stronę wieje wiatr raczej nie powinno mieć wpływu na warunki drogowe, ale wstępnie zostawimy i później potwierdzimy lub obalimy swoje założenia w oparciu o konkretną metrykę
- `wilgotnosc` - może mieć istotny wpływ na warunki drogowe, zostawiamy
- `suma_opadu` - może mieć istotny wpływ na warunki drogowe, zostawiamy
- `cisnienie` - nie wydaje się mieć bezpośredniego wpływu na warunki drogowe, ale pośredni wpływ może być warty zbadania (np. poprzez wpływ na samopoczucie kierowców)
- `timestamp` - podstawa jakichkolwiek wykresów/zestawień, dodatkowo jest to jedyna kolumna w tym pliku umożliwiająca łączenie rekordów z pozostałymi plikami, zostawiamy

### `delays-merged.csv`
- `Type` - można przygotować zestawienia pomiędzy autobusami i tramwajami, zostawiamy
- `Vehicle No` - można sprawdzić czy konkretne pojazdy są bardziej lub mniej skłonne do spóźnień, zostawiamy
- `Brigade No` - podobnie jak poprzednia kolumna, można sprawdzić czy jakieś konkretne brygady są bardziej lub mniej skłonne do spóźnień, zostawiamy
- `Route` - ma odzwierciedlenie również w feedzie gtfs, przydatne jako identyfikator do powiązania ze sobą rekordów, zostawiamy
- `Trip Headsign` - pozwala na wprowadzenie dodatkowej kolumny za pomocą połączenia z kolumną `Route`, np. `179_Os. Kabaty`, `179_Ursynów Płn`. Dodatkowa kategoria do analizy, zostawiamy.
- `Delay` - kluczowa dana w tym pliku, zostawiamy
- `Stop Name` - ma odzwierciedlenie również w feedzie gtfs, przydatne jako identyfikator do powiązania ze sobą rekordów, zostawiamy
- `Outside` - to czy pojazd jest poza trasą wydaje się mieć istotny wpływ na spóźnienia, ciekawa dana do analizy, zostawiamy
- `Timestamp` - podstawa jakichkolwiek wykresów/zestawień, zostawiamy

### `gtfs/stop_times.txt`
W tym pliku interesuje nas jedynie informacja o tym, czy dany przystanek jest na żądanie. Potrzebujemy więc wykorzystać plik `stops.txt` do zmapowania kolumny `stop_id` na nazwy przystanków, a 2 kolumny: `pickup_type` i `drop_off_type` można znormalizować do pojedynczej kolumny `on_demand`, która będzie miała wartość typu `bool`. Ten plik wnosi nam więc tylko jedną kolumnę z danymi - nie ma sensu trzymać tego jako oddzielny plik, kolumnę `on_demand` dokleimy po prostu do pliku `stops.txt`.

### `gtfs/stops.txt`
- `stop_id` - potrzebujemy etykiety do łączenia z danymi o opóźnieniach, a ta kolumna występuje tylko w gtfs, po wykorzystaniu do doklejenia danych z `stop_times.txt` usuwamy
- `stop_name` - etykieta do łączenia danych z innymi plikami
- `stop_lat` - istotne dane o fizycznej lokalizacji przystanku
- `stop_lon` - istotne dane o fizycznej lokalizacji przystanku